In [1]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('../desdeo_emo')))

In [2]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

from desdeo_problem.Variable import variable_builder
from desdeo_problem.Objective import _ScalarObjective
from desdeo_problem.Problem import MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from pygmo import fast_non_dominated_sorting as nds
from desdeo_emo.othertools.ReferenceVectors import ReferenceVectors



from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_

In [3]:
import baseADM
from baseADM import *
import generatePreference as gp
import visual
from visual import *

## Coello MOP7

### Definition
![Definition](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/docs/notebooks/notebook_extras/MOP7.png)

### Pareto set and front
![Front](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/docs/notebooks/notebook_extras/MOP7_PF.png)

## Define objective functions

In [4]:
def f_1(x):
    term1 = ((x[:,0] - 2) ** 2) / 2
    term2 = ((x[:,1] + 1) ** 2) / 13
    return term1 + term2 + 3

def f_2(x):
    term1 = ((x[:, 0] + x[:, 1] - 3) ** 2) / 36
    term2 = ((-x[:, 0] + x[:, 1] + 2) ** 2) / 8
    return term1 + term2 - 17

def f_3(x):
    term1 = ((x[:, 0] + (2 * x[:, 1]) - 1) ** 2) / 175
    term2 = ((-x[:, 0] + 2* x[:, 1]) ** 2) / 17
    return term1 + term2 - 13

## Create Variable objects

In [5]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-400, -400],
                             upper_bounds=[400, 400])
list_vars

## Create Objective objects

In [6]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)
f3 = _ScalarObjective(name='f3', evaluator=f_3)
list_objs = [f1, f2, f3]

## Create the problem object

In [7]:
problem = MOProblem(variables=list_vars, objectives=list_objs)

## Using the EAs

Pass the problem object to the EA, pass parameters as arguments if required.

In [8]:
evolver = NSGAIII(problem,
                  n_iterations=3,
                  n_gen_per_iter=50)
evolver2 = RVEA(problem,
                  n_iterations=3,
                  n_gen_per_iter=50)

In [9]:
plot, pref = evolver.requests()
plot2, pref2 = evolver2.requests()

In [10]:
pref
pref2

In [11]:
while evolver.continue_evolution():
    evolver.iterate()
while evolver2.continue_evolution():
    evolver2.iterate()

## Extracting optimized decision variables and objective values

In [12]:
individuals = evolver.population.individuals

fig1 = go.Figure(data=go.Scatter(x=individuals[:,0], y=individuals[:,1], mode="markers"))
fig1

In [13]:
objectives = evolver.population.objectives


In [14]:
objectives2 = evolver2.population.objectives
#print(objectives2)

In [15]:
cf = generate_composite_front(objectives, objectives2)
fig2 = go.Figure(data=go.Scatter3d(x=cf[:,0],
                                   y=cf[:,1],
                                   z=cf[:,2],
                                   mode="markers",
                                   marker_size=2))
fig2

In [16]:
lattice_resolution = 4
reference_vectors = ReferenceVectors(lattice_resolution, problem.n_of_objectives)

In [17]:
base = baseADM(cf, reference_vectors)

In [18]:
visualize_3D_front_rvs(base.normalized_front, base.vectors)

In [19]:
rp = gp.generateRP4learning(base)
visualize_3D_front_rp(base.composite_front, rp)

In [20]:
rp

array([  3.08128374, -16.92034159, -12.84117366])

In [21]:
max_assigned_vector = gp.get_max_assigned_vector(base.assigned_vectors)
rp2 = gp.generateRP4decision(base, max_assigned_vector)
visualize_3D_front_rp(base.composite_front, rp2)

In [22]:
rp2

array([  3.18277198, -16.99975275, -12.9396962 ])